# imports

In [14]:
import pandas as pd
import numpy as np
from sympy import *
import cylp.cy.CyClpSimplex as LP
import scipy.sparse as sp

# data preparation

In [15]:
knapsack = pd.read_csv('knapsack_0.csv', sep=" ", names=['item', 'prize', 'weight'])
knapsack

,item,prize,weight
0,Laptop,10,0.50
1,Flashcard,5,0.01
2,Cryptokey,1,0.01
3,Lamp,1,5.00
4,Battery,1,0.05
5,Charger,0,0.01


In [16]:
def dependencies_parser(lines):
    result = ''
    lines = [line.strip().split() for line in lines]
    for line in lines:
        if len(line) > 2:
            result += ' AND' + ' (' + ' OR '.join(line[0:-1]) + ') -> ' + line[-1] 
        else:
            result += ' AND ' + line[0] + ' -> ' + line[1] 

    return result[5:]

In [17]:
def dependencies_to_dir(lines):
    result = {}
    lines = [line.strip().split() for line in lines]
    for line in lines:
        result[line[-1]] = line[0:-1]
    return result

In [18]:
with open("dependencies.txt") as f:
    # dependencies = dependencies_parser(f.readlines())
    dependencies = [line.strip().split() for line in f.readlines()]
dependencies

[['Cryptokey', 'Flashcard'],
 ['Flashcard', 'Laptop'],
 ['Battery', 'Charger', 'Laptop']]

In [19]:
knapsack_vol = pd.read_csv('knapsack.csv', sep=" ", names=['item', 'prize', 'weight', 'volume'])
knapsack_vol

,item,prize,weight,volume
0,Laptop,10,0.50,0.20
1,Flashcard,5,0.01,0.02
2,Cryptokey,1,0.01,0.02
3,Lamp,1,5.00,1.00
4,Battery,1,0.05,0.06
5,Charger,0,0.01,0.06


In [20]:
oversized = pd.read_csv('oversized.csv', sep=" ", names=['item', 'volume'])
oversized = {oversized.item[i] : oversized.volume[i] for i in range(len(oversized))}
oversized

{'Laptop': 0.05, 'Lamp': 0.01}

# knapsack problem

In [21]:
# solver
def solve_knapsack(knapsack, W):
    p, w, n = np.array(knapsack.prize), np.array(knapsack.weight), len(np.array(knapsack.prize))
    model = LP()
    x = model.addVariable('x', n, isInt=True)
    model.objective = -p
    model += sp.eye(n) * x >= np.zeros(n)
    model += sp.eye(n) * x <= np.ones(n)
    model += np.matrix(w) * x <= W
    cbcModel = model.getCbcModel()
    cbcModel.logLevel = False
    cbcModel.solve()
    solution = np.array(cbcModel.primalVariableSolution['x'].round()).astype(int) 
    return ' '.join([knapsack.item[i] if v==1 else '' for i, v in enumerate(solution)])

In [27]:
# results
for i in np.arange(0, 0.11, 0.01):
    print(str(i), ':\t', solve_knapsack(knapsack, i))
print('...')
for i in np.arange(0.49, 0.511, 0.01):
    print(str(i), ':\t', solve_knapsack(knapsack, i))
print('...')
print(0.53, ':\t', solve_knapsack(knapsack, 0.53))
print('...')
print(0.57, ':\t', solve_knapsack(knapsack, 0.57))
print('...')
for i in np.arange(4.99, 5.011, 0.01):
    print(str(i), ':\t', solve_knapsack(knapsack, i))
print('...')
print(6.0, ':\t', solve_knapsack(knapsack, 6.0))


0.0 :	      
0.01 :	  Flashcard    
0.02 :	  Flashcard Cryptokey   
0.03 :	  Flashcard Cryptokey   
0.04 :	  Flashcard Cryptokey   
0.05 :	  Flashcard Cryptokey   
0.06 :	  Flashcard Cryptokey   
0.07 :	  Flashcard Cryptokey  Battery 
0.08 :	  Flashcard Cryptokey  Battery 
0.09 :	  Flashcard Cryptokey  Battery 
0.1 :	  Flashcard Cryptokey  Battery 
...
0.49 :	  Flashcard Cryptokey  Battery 
0.5 :	 Laptop     
0.51 :	 Laptop Flashcard    
...
0.53 :	 Laptop Flashcard Cryptokey   
...
0.57 :	 Laptop Flashcard Cryptokey  Battery 
...
4.99 :	 Laptop Flashcard Cryptokey  Battery 
5.0 :	 Laptop Flashcard Cryptokey  Battery 
5.01 :	 Laptop Flashcard Cryptokey  Battery 
...
6.0 :	 Laptop Flashcard Cryptokey Lamp Battery 


# knapsack problem with dependencies

In [23]:
# solver
def solve_knapsack_dep(knapsack, W, dependencies):
    p, w, items = np.array(knapsack.prize), np.array(knapsack.weight), list(knapsack.item)
    n = len(p)
    model = LP()
    x = model.addVariable('x', n, isInt=True)
    model.objective = -p
    model += sp.eye(n) * x >= np.zeros(n)
    model += sp.eye(n) * x <= np.ones(n)
    model += np.matrix(w) * x <= W
    for d in dependencies:
        model += sum([x[items.index(i)] for i in d[0:-1]]) - x[items.index(d[-1])] >= 0
    cbcModel = model.getCbcModel()
    cbcModel.logLevel = False
    cbcModel.solve()
    solution = np.array(cbcModel.primalVariableSolution['x'].round()).astype(int) 
    return ' '.join([knapsack.item[i] if v==1 else '' for i, v in enumerate(solution)])


In [24]:
# results
for i in np.arange(0, 0.11, 0.01):
    print(str(i), ':\t', solve_knapsack_dep(knapsack, i, dependencies))
print('...')
for i in np.arange(0.49, 0.511, 0.01):
    print(str(i), ':\t', solve_knapsack_dep(knapsack, i, dependencies))
print('...')
print(0.53, ':\t', solve_knapsack_dep(knapsack, 0.53, dependencies))
print('...')
print(0.57, ':\t', solve_knapsack_dep(knapsack, 0.57, dependencies))
print('...')
for i in np.arange(4.99, 5.011, 0.01):
    print(str(i), ':\t', solve_knapsack_dep(knapsack, i, dependencies))
print('...')
print(6.0, ':\t', solve_knapsack_dep(knapsack, 6.0, dependencies))
print('...')

0.0 :	      
0.01 :	   Cryptokey   
0.02 :	  Flashcard Cryptokey   
0.03 :	  Flashcard Cryptokey   
0.04 :	  Flashcard Cryptokey   
0.05 :	  Flashcard Cryptokey   
0.06 :	  Flashcard Cryptokey   
0.07 :	  Flashcard Cryptokey  Battery 
0.08 :	  Flashcard Cryptokey  Battery 
0.09 :	  Flashcard Cryptokey  Battery 
0.1 :	  Flashcard Cryptokey  Battery 
...
0.49 :	  Flashcard Cryptokey  Battery 
0.5 :	  Flashcard Cryptokey  Battery 
0.51 :	  Flashcard Cryptokey  Battery 
...
0.53 :	 Laptop Flashcard Cryptokey   Charger
...
0.57 :	 Laptop Flashcard Cryptokey  Battery 
...
4.99 :	 Laptop Flashcard Cryptokey  Battery 
5.0 :	 Laptop Flashcard Cryptokey  Battery 
5.01 :	 Laptop Flashcard Cryptokey  Battery 
...
6.0 :	 Laptop Flashcard Cryptokey Lamp Battery Charger
...


# knapsack problem with oversize items

In [25]:
# solver
def solve_knapsack_vol(knapsack, W, oversized):
    p, w, v = np.array(knapsack.prize), np.array(knapsack.weight), np.array(knapsack.volume)
    items = list(knapsack_vol.item)
    n = len(p)
    model = LP()
    x = model.addVariable('x', n, isInt=True)
    y = {}
    for key in oversized:
        y[key] = model.addVariable('y'+key, n, isInt=True)
    
    model.objective = -p
    model += sp.eye(n) * x >= np.zeros(n)
    model += sp.eye(n) * x <= np.ones(n)
    for key in oversized:
        model += sp.eye(n) * y[key] >= np.zeros(n)
        model += sp.eye(n) * y[key] <= np.ones(n)
    model += np.matrix(w) * x <= W
    
    for oitem, ovolume in oversized.items():
        for i in range(n):
            model += x[i] + x[items.index(oitem)] - y[oitem][i] <= 1
            if i != items.index(oitem):
                model += v[i] * y[oitem][i] <= ovolume

    cbcModel = model.getCbcModel()
    cbcModel.logLevel = False
    cbcModel.solve()
    solution = np.array(cbcModel.primalVariableSolution['x'].round()).astype(int) 
    return ' '.join([knapsack.item[i] if v==1 else '' for i, v in enumerate(solution)])

In [26]:
# results
for i in np.arange(0, 0.11, 0.01):
    print(str(i), ':\t', solve_knapsack_vol(knapsack_vol, i, oversized))
print('...')
for i in np.arange(0.49, 0.511, 0.01):
    print(str(i), ':\t', solve_knapsack_vol(knapsack_vol, i, oversized))
print('...')
print(0.53, ':\t', solve_knapsack_vol(knapsack_vol, 0.53, oversized))
print('...')
print(0.57, ':\t', solve_knapsack_vol(knapsack_vol, 0.57, oversized))
print('...')
for i in np.arange(4.99, 5.011, 0.01):
    print(str(i), ':\t', solve_knapsack_vol(knapsack_vol, i, oversized))
print('...')
print(6.0, ':\t', solve_knapsack_vol(knapsack_vol, 6.0, oversized))

0.0 :	      
0.01 :	  Flashcard    
0.02 :	  Flashcard Cryptokey   
0.03 :	  Flashcard Cryptokey   
0.04 :	  Flashcard Cryptokey   
0.05 :	  Flashcard Cryptokey   
0.06 :	  Flashcard Cryptokey   
0.07 :	  Flashcard Cryptokey  Battery 
0.08 :	  Flashcard Cryptokey  Battery 
0.09 :	  Flashcard Cryptokey  Battery 
0.1 :	  Flashcard Cryptokey  Battery 
...
0.49 :	  Flashcard Cryptokey  Battery 
0.5 :	 Laptop     
0.51 :	 Laptop Flashcard    
...
0.53 :	 Laptop Flashcard Cryptokey   
...
0.57 :	 Laptop Flashcard Cryptokey   
...
4.99 :	 Laptop Flashcard Cryptokey   
5.0 :	 Laptop Flashcard Cryptokey   
5.01 :	 Laptop Flashcard Cryptokey   
...
6.0 :	 Laptop Flashcard Cryptokey   
